# mHSP2: Set the "Save" Parameters to HDF5

**mHSP2** is a reduced and modified version of [HSPsquared](https://github.com/respec/HSPsquared). **mHSP2** is reduced because it will only simulate the movement of water with *PERLND*, *IMPLND*, and *RCHRES* targets or simulation structures. It is modified relative to *HSPsquared* (or HSP2) so that it can be dynamically coupled to MODFLOW 6 using a daily time step.

Dynamic coupling means that **mHSP2** runs one simulation day, then passes deep infiltration and stream losses to groundwater to MODFLOW 6, and finally MODFLOW 6 simulates the same day, using the inputs from **mHSP2**. At the end of the day, MODFLOW 6 sends discharge to the ground surface information back to **mHSP2** which is waiting for this information. Once **mHSP2** receives this information, it simulates the next day. The two programs alternate active simulation for every day during the simulation period.

A goal in **mHSP2** design was to ensure that the user could use whatever, desired variant of HSPF for standalone watershed and streamflow routing model creation and calibration. Then, the HSPF model could be converted for use in **mHSP2** for integrated hydrologic modeling of the study site. **mHSP2** uses the input formats of *HSPsquared*. An advantage of *HSPsquared* is that it uses a single [HDF5](https://support.hdfgroup.org/HDF5/whatishdf5.html) file for model inputs and outputs. *HSPsquared* also provides conversion utilities to go from the historical HSPF inputs of a fixed format text user control input, .uci, file and a binary watershed data management, .wdm, time series input archive to the single HDF5 file. **mHSP2** relies on the translation utilities that are part of *HSPsquared* to provide the capability for a user to convert any HSPF model to the inputs for the **mHSP2** portion of an coupled simulation.

**mHSP2** was designed and created for application to only a small subset of the typical HSPF model use cases. Consequently, there are some specific requirements for **mHSP2** model configuration. The relevant requirement for this Notebook is that **mHSP2** uses multiple *RCHRES* exits to route water to:

1. Downstream to the next *RCHRES*
2. Losses to groundwater which are routed to the MODFLOW 6, unsaturated zone flow (UZF), boundary condition package

Given the importance of *RCHRES* exits in this formulation, development of **mHSP2** included special focus on the treatment of multiple *RCHRES* exits in *HSPsquared*. There seem to be some limitations in *HSPsquared* relative to other HSPF versions regarding the use of multiple exits. Consequently, **mHSP2** development focused on reinstating the original HSPF logic related to multiple *RCHRES* exits. One of these limitations is that it is not possible in *HSPsquared* to specify saving of outflows by exit.

The purpose of this Notebook is to provide the **mHSP2** user with a template for modifying the HDF5 file produced by *HSPsquared* so that the user can customize the **mHSP2** outputs to include outputs from individual exits for each *RCHRES* in a simulation.

The customization of the *HSPsquared* HDF5 file is further complicated by the existence of two versions of *HSPsquared*, each of which uses an HDF5 file with a different format. These two versions are:

1. New, or modified, format: [Python 3.6+ HSPsquared](https://github.com/respec/HSPsquared)
2. Old, or original, format: [Python 2 HSPsquared](https://github.com/respec/HSPsquared/tree/archivePy2)

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import h5py
import pandas as pd
import numpy as np
from IPython.display import display, HTML

HDF5 file for modification - change this filename and path to modify a different file

In [3]:
HDF5_FILE = r'C:\Users\nmartin\Documents\LOCA\Test_Models\HSP2\DC_Subs.h5'
#HDF5_FILE = r'C:\Users\nmartin\Documents\LOCA\Test_Models\HSP2_NewFormat\IDC_NewFormatTest.h5'

**mHSP2** supported output parameters - Do **NOT** change these unless you change the **mHSP2** code

In [4]:
PERLND_OUTPUTs = [ "AGWET", "AGWI", "AGWO", "AGWS", "BASET", "CEPE", 
                     "CEPS", "GWVS", "IFWI", "IFWO", "IFWS", "IGWI",
                     "INFFAC", "INFIL", "LZET", "LZI", "LZS", "RPARM", 
                     "PERC", "PERO", "PERS", "PET", "SUPY", 
                     "SURI", "SURO", "SURS", "TAET", "TGWS", "UZET", 
                     "UZI", "UZS" ]

In [5]:
IMPLND_OUTPUTs = [ "IMPEV", "IMPS", "PET", "RETS", "SUPY", "SURI", 
                     "SURO", "SURS" ]

In [6]:
RCHRES_OUTPUTs = [ "AVDEP", "AVVEL", "DEP", "HRAD", "IVOL", "O1", 
                     "O2", "O3", "O4", "O5", "OVOL1", "OVOL2", 
                     "OVOL3", "OVOL4", "OVOL5", "PRSUPY", "RO", 
                     "ROVOL", "SAREA", "STAGE", "VOL", "TAU", 
                     "TWID", "USTAR", "VOLEV" ]

## Determine the HDF5 Format

In the code block below, determine which type of HDF5 file that are working with so that know where to put the "SAVE" information.

The **HDF_FMT** variable is used to store the format style.

- **HDF_FMT** == 0 means old or original format
- **HDF_FMT** == 1 means new or modified format

In [7]:
GTABLE_PATH = '/CONTROL/GLOBAL/table'
UNITS_STR = 'units'
OldFmt = 0
NewFmt = 1
# locals
FoundUnits = False
# start
chkFile = h5py.File(HDF5_FILE, 'r')
globs = chkFile[GTABLE_PATH]
NRows = globs.shape[0]
for iI in range(NRows):
    testStr = globs[iI][0]
    utStr = testStr.decode()
    if utStr == UNITS_STR:
        FoundUnits = True
        break
    # end if
# end for
# close the file
chkFile.close()
# set the units
if FoundUnits:
    HDF_FMT = OldFmt
else:
    HDF_FMT = NewFmt
# end if
if HDF_FMT == 0:
    print("Old or original format HDF5 file")
else:
    print("New or modified format HDF5 file")

Old or original format HDF5 file


## Old Format Modifications

Use this section if **HDF_FMT** == 0 and you have an old format file

First get the inventory of targets, or simulation structures. For **mHSP2**, these are limited to *PERLND*, *IMPLND*, and *RCHRES*.

In [8]:
with pd.HDFStore( HDF5_FILE ) as store:
    opsDF = store.get( "/CONTROL/OP_SEQUENCE/" )
# end with and close HDF5 file

In [9]:
display( HTML( opsDF.to_html() ) )

,TARGET,ID,DELT
0,PERLND,P001,1440
1,PERLND,P002,1440
2,PERLND,P003,1440
3,PERLND,P004,1440
4,PERLND,P005,1440
5,IMPLND,I001,1440
6,IMPLND,I002,1440
7,IMPLND,I003,1440
8,IMPLND,I004,1440
9,IMPLND,I005,1440


In [10]:
TARG_DICT = dict()
for index, row in opsDF.iterrows():
    cKey = str( row['TARGET'] )
    cVal = str( row['ID'] )
    if cKey in TARG_DICT.keys():
        TARG_DICT[cKey].append( cVal )
    else:
        TARG_DICT[cKey] = [ cVal ]
    # end if
# end for

In [11]:
TARG_DICT

{'PERLND': ['P001', 'P002', 'P003', 'P004', 'P005'],
 'IMPLND': ['I001', 'I002', 'I003', 'I004', 'I005'],
 'RCHRES': ['R001',
  'R006',
  'R002',
  'R007',
  'R003',
  'R008',
  'R004',
  'R009',
  'R005']}

Because **mHSP2** only supports *PERLND*, *IMPLND*, and *RCHRES* only go through these structures in TARG_DICT to set our "SAVE" values.

In [12]:
GOOD_TARGS = [ 'PERLND', 'IMPLND', 'RCHRES' ]

Do these one at a time starting with *PERLND*

### PERLND

In [13]:
tTarg = GOOD_TARGS[0]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

Build a Pandas DataFrame to put into the HDF5 file and overwriting the existing "Save" information. Here all fields are set to 1 which means "Save" or output. You can edit the fields individually using HDFView or change the logic here to only set 1 for the what you want output.

In [14]:
DataDict = dict()
for tHdr in PERLND_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [15]:
display( HTML( woDF.to_html() ) )

,AGWET,AGWI,AGWO,AGWS,BASET,CEPE,CEPS,GWVS,IFWI,IFWO,IFWS,IGWI,INFFAC,INFIL,LZET,LZI,LZS,RPARM,PERC,PERO,PERS,PET,SUPY,SURI,SURO,SURS,TAET,TGWS,UZET,UZI,UZS
P001,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
P002,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
P003,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
P004,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
P005,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/PERLND/PWATER/SAVE"]
# end with

In [17]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/PERLND/PWATER/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 

### IMPLND

In [18]:
tTarg = GOOD_TARGS[1]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

In [19]:
DataDict = dict()
for tHdr in IMPLND_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [20]:
display( HTML( woDF.to_html() ) )

,IMPEV,IMPS,PET,RETS,SUPY,SURI,SURO,SURS
I001,1,1,1,1,1,1,1,1
I002,1,1,1,1,1,1,1,1
I003,1,1,1,1,1,1,1,1
I004,1,1,1,1,1,1,1,1
I005,1,1,1,1,1,1,1,1


In [21]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/IMPLND/IWATER/SAVE"]
# end with

In [22]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/IMPLND/IWATER/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 

### RCHRES

In [23]:
tTarg = GOOD_TARGS[2]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

In [24]:
DataDict = dict()
for tHdr in RCHRES_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [25]:
display( HTML( woDF.to_html() ) )

,AVDEP,AVVEL,DEP,HRAD,IVOL,O1,O2,O3,O4,O5,OVOL1,OVOL2,OVOL3,OVOL4,OVOL5,PRSUPY,RO,ROVOL,SAREA,STAGE,VOL,TAU,TWID,USTAR,VOLEV
R001,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R002,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R003,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R004,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R005,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R006,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R007,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R008,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
R009,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [26]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/RCHRES/HYDR/SAVE/"]
# end with

In [27]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/RCHRES/HYDR/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 

## New Format Modifications

Use this section if **HDF_FMT** == 1 and you have an new format file

First get the inventory of targets, or simulation structures. For **mHSP2**, these are limited to *PERLND*, *IMPLND*, and *RCHRES*.

In [ ]:
with pd.HDFStore( HDF5_FILE ) as store:
    opsDF = store.get( "/CONTROL/OP_SEQUENCE/" )
# end with and close HDF5 file

In [ ]:
display( HTML( opsDF.to_html() ) )

In [ ]:
TARG_DICT = dict()
for index, row in opsDF.iterrows():
    cKey = str( row['OPERATION'] )
    cVal = str( row['SEGMENT'] )
    if cKey in TARG_DICT.keys():
        TARG_DICT[cKey].append( cVal )
    else:
        TARG_DICT[cKey] = [ cVal ]
    # end if
# end for

In [ ]:
TARG_DICT

Because **mHSP2** only supports *PERLND*, *IMPLND*, and *RCHRES* only go through these structures in TARG_DICT to set our "SAVE" values.

In [ ]:
GOOD_TARGS = [ 'PERLND', 'IMPLND', 'RCHRES' ]

Do these one at a time starting with *PERLND*

### PERLND

In [ ]:
tTarg = GOOD_TARGS[0]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

Build a Pandas DataFrame to put into the HDF5 file and overwriting the existing "Save" information. Here all fields are set to 1 which means "Save" or output. You can edit the fields individually using HDFView or change the logic here to only set 1 for the what you want output.

In [ ]:
DataDict = dict()
for tHdr in PERLND_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [ ]:
display( HTML( woDF.to_html() ) )

In [ ]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/PERLND/PWATER/SAVE"]
# end with

In [ ]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/PERLND/PWATER/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 

### IMPLND

In [ ]:
tTarg = GOOD_TARGS[1]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

In [ ]:
DataDict = dict()
for tHdr in IMPLND_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [ ]:
display( HTML( woDF.to_html() ) )

In [ ]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/IMPLND/IWATER/SAVE"]
# end with

In [ ]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/IMPLND/IWATER/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 

### RCHRES

In [ ]:
tTarg = GOOD_TARGS[2]
tIDs = sorted( TARG_DICT[tTarg ] )
NumIDs = len( tIDs )
Vals = np.ones( NumIDs, dtype=np.int32 )

In [ ]:
DataDict = dict()
for tHdr in RCHRES_OUTPUTs:
    DataDict[tHdr] = Vals
# end for
woDF = pd.DataFrame( index=tIDs, data=DataDict )

In [ ]:
display( HTML( woDF.to_html() ) )

In [ ]:
with h5py.File(HDF5_FILE, 'a') as h5:
    del h5["/RCHRES/HYDR/SAVE/"]
# end with

In [ ]:
with pd.HDFStore( HDF5_FILE ) as store:
    store.put( key="/RCHRES/HYDR/SAVE/", value=woDF, format='table', index=True, data_columns=True )
# end with 